In [ ]:
import cv2
import sys
import pandas as pd



In [ ]:
video_path=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs preprocess\JOMR_nov25_BSD_01.mp4'
play_speed=1.0
team1_name="JOMR"
team2_name="MyH_HeR"
output_dir= r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\points_segments'

temp_list=cv2_point_segment_cut(
    video_path=video_path,
    play_speed=play_speed,
    team1_name=team1_name,
    team2_name=team2_name,
    output_dir=output_dir
)

In [6]:
# from montage_auto_frames_gpu import cv2_temp_list_to_operate
import pandas as pd
import cv2
import sys

def cv2_point_segment_cut(
        video_path : str,
        play_speed : float = 1.0,
        team1_name: str = "JOMR",
        team2_name: str = "adversaire",
        output_dir: str = None
    ) -> pd.DataFrame:

    """
    Création d'un dataframe contenant les segments de points extraits d'une vidéo de match, avec les informations sur le score et les équipes.
    A utiliser ensuite avec cut_point_gpu pour découper les segments de points à partir de ce dataframe. 
    Permet également de contrôler la lecture de la vidéo (pause, vitesse) pour faciliter l'identification des points.
    
    Args:
        video_path (str): Chemin vers la vidéo à traiter.
        play_speed (float, optional): Vitesse de lecture de la vidéo (1=normale, 0=pause, >1=plus rapide). Par défaut à 1.0.
        team1_name (str, optional): Nom de l'équipe 1. Par défaut à "JOMR".
        team2_name (str, optional): Nom de l'équipe 2. Par défaut à "adversaire".
        output_dir (str, optional): Dossier de sortie pour les segments extraits. Si None, les segments seront extraits dans le même dossier que la vidéo source.
    Returns:
        DataFrame contenant les informations sur les segments de points extraits, avec les colonnes : 
        'point_index',
        'action',
        'start_frame',
        'end_frame',
        'score_team1',
        'score_team2',
        'set_team1',
        'set_team2'
    """
    # Initialisation des variables
    temp_list = list()
    last_action = None

    # Mapping des touches aux temp_list
    key_action_map = {
        ord('0'): 'debut du set',
        ord('1'): f'service {team1_name}',
        ord('3'): f'service {team2_name}',
        ord('2'): 'fin point',
        ord('5'): '*SWITCH*',
        ord('8'): 'Temps mort',
        }


    # Afficher les touches disponibles en overlay sur la vidéo
    help_lines = [
        "0 : debut du set",
        f"1 : service {team1_name}",
        f"3 : service {team2_name}",
        "2 : fin du point",
        "5 : switch",
        "8 : temps mort"
    ]



    _orig_imshow = cv2.imshow

    def _imshow_with_help(winname, frame):
        if frame is not None:
            x, y = 30, 120
            for i, line in enumerate(help_lines):
                cv2.putText(frame,
                            line,
                            (x, y + i * 25),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            (255, 255, 255),
                            2,
                            cv2.LINE_AA)
        _orig_imshow(winname, frame)

    cv2.imshow = _imshow_with_help


    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)

    def _waitKey_fast(ms):
        # Réduire le délai proportionnellement à la vitesse (au moins 1 ms)
        adj = max(1, int(ms / play_speed))
        return cv2.waitKey(adj)

    # Vérifier que la vidéo est bien ouverte
    if not cap.isOpened():
        print("Erreur : impossible d’ouvrir la vidéo.")
        sys.exit()

    # Récupérer les FPS pour convertir les frames en temps
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_number = 0

    # État de pause et rotation
    paused = False


    try:
        while cap.isOpened():

            # Lire une frame seulement si on n'est pas en pause
            if not paused:
                ret, frame = cap.read()

                # Arrêter si fin de vidéo ou erreur
                if not ret:
                    print("Fin de la vidéo ou erreur de lecture.")
                    break

                # Incrémenter le compteur de frames
                frame_number += 1

            # Affiche la dernière action
            if last_action and ret:
                cv2.putText(frame,
                            f"Derniere action : {last_action}",
                            (30, 40),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 255, 0),
                            2,
                            cv2.LINE_AA)

            # Indiquer le mode pause sur l'image affichée
            if paused and ret:
                cv2.putText(frame,
                            "|| PAUSE ||",
                            (30, 80),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 0, 255),
                            2,
                            cv2.LINE_AA)

            # Afficher la frame courante
            if ret:
                cv2.imshow(f'{video_path}', frame)

            # Gestion des entrées clavier
            key = _waitKey_fast(30) & 0xFF
            if key == ord('q'):  # quitter
                break
            elif key == ord(' '):  # pause/reprise
                paused = not paused
            elif key == ord('+'):  # augmenter la vitesse
                play_speed += 0.5
                continue
            elif key == ord('-'):  # diminuer la vitesse
                play_speed = max(0.5, play_speed - 0.5)
                continue
            elif key in key_action_map: # enregistrer l'action associée à la touche, avec le numéro de frame
                action_name = key_action_map[key]
                last_action = action_name
                temp_list.append({
                    'Frame': frame_number,
                    'Action': action_name
                })
            elif key == ord('7'): # erreur de codage, revenir en arrière
                if temp_list:
                    removed_action = temp_list.pop()
                    print(f"Action supprimée : {removed_action}")
                    last_action = temp_list[-1]['Action'] if temp_list else None
                else:
                    print("Aucune action à supprimer.")
                # Revenir à la frame de l'action supprimée et mettre la lecture en pause
                if temp_list:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, temp_list[-1]['Frame'])
                    frame_number = temp_list[-1]['Frame']
                    paused = True
                else:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                    frame_number = 0



    finally:
        # Libérer les ressources OpenCV
        cap.release()
        cv2.destroyAllWindows()


    # print(temp_list)

    # Initialiser une liste pour stocker les segments de points
    list_actions = list()

    i = 0
    while i < len(temp_list):
        action = temp_list[i]
        

        # Si l'action n'est pas 'fin point', on la traite
        if action['Action'] != 'fin point':
            start_frame = action['Frame']
            service_side = action['Action']
            # Chercher le prochain 'fin point'
            end_frame = None
            j = i + 1
            while j < len(temp_list):
                if temp_list[j]['Action'] == 'fin point':
                    end_frame = temp_list[j]['Frame']
                    break
                j += 1
            
            # Ajouter à la liste si on a trouvé un 'fin point'
            if end_frame is not None:
                list_actions.append({
                    'Service_side': service_side,
                    'Start_frame': start_frame,
                    'End_frame': end_frame
                })
        
        i += 1

    # Convertir en DataFrame pandas
    df_points = pd.DataFrame(list_actions)

    # Ajouter les colonnes de score
    df_points[f'{team1_name}_score'] = 0
    df_points[f'{team2_name}_score'] = 0
    df_points[f'{team1_name}_sets'] = 0
    df_points[f'{team2_name}_sets'] = 0


    # Mettre à jour les scores en fonction du service_side
    for idx, row in df_points.iterrows():
        if df_points['Service_side'].iloc[idx] == 'debut du set':
            # Première ligne : initialiser selon le service
            if row['Service_side'] == f'service {team1_name}':
                df_points.at[idx, f'{team1_name}_score'] = 1
                df_points.at[idx, f'{team2_name}_score'] = 0
            elif row['Service_side'] == f'service {team2_name}':
                df_points.at[idx, f'{team1_name}_score'] = 0
                df_points.at[idx, f'{team2_name}_score'] = 1
        else:
            # Lignes suivantes : reprendre le score précédent et incrémenter selon le service
            df_points.at[idx, f'{team1_name}_score'] = df_points.at[idx - 1, f'{team1_name}_score']
            df_points.at[idx, f'{team2_name}_score'] = df_points.at[idx - 1, f'{team2_name}_score']
            
            if row['Service_side'] == f'service {team1_name}':
                df_points.at[idx, f'{team1_name}_score'] += 1
            elif row['Service_side'] == f'service {team2_name}':
                df_points.at[idx, f'{team2_name}_score'] += 1

    # Mettre à jour les scores de sets au début de chaque nouveau set
    for idx, row in df_points.iterrows():
        if row['Service_side'] == 'debut du set' and idx > 0:
            # Comparer les scores du set précédent
            prev_score_team1 = df_points.at[idx - 1, f'{team1_name}_score']
            prev_score_team2 = df_points.at[idx - 1, f'{team2_name}_score']
            
            # Récupérer les sets précédents
            df_points.at[idx, f'{team1_name}_sets'] = df_points.at[idx - 1, f'{team1_name}_sets']
            df_points.at[idx, f'{team2_name}_sets'] = df_points.at[idx - 1, f'{team2_name}_sets']
            
            # Ajouter un set au gagnant
            if prev_score_team1 > prev_score_team2:
                df_points.at[idx, f'{team1_name}_sets'] += 1
            else:
                df_points.at[idx, f'{team2_name}_sets'] += 1
        elif idx > 0:
            # Pour les autres lignes, conserver le nombre de sets
            df_points.at[idx, f'{team1_name}_sets'] = df_points.at[idx - 1, f'{team1_name}_sets']
            df_points.at[idx, f'{team2_name}_sets'] = df_points.at[idx - 1, f'{team2_name}_sets']

    return list_actions, df_points

# -------------------------------------------------------------------------------------------------------------




In [18]:
import pandas as pd
test_df_points = pd.read_csv(r'C:\Users\habib\Documents\GitHub\DataBeach\test_df_points_JOMR_nov25_BSD_01.csv')

test_df_points.tail(5)
# test_df_points[test_df_points['Service_side']=='*SWITCH*']


,Service_side,Start_frame,End_frame,JOMR_score,adversaire_score,JOMR_sets,adversaire_sets,point_index
51,service JOMR,34125,34468,12,6,1,0,43.0
52,service adversaire,34919,35227,12,7,1,0,44.0
53,*SWITCH*,35461,35756,12,7,1,0,NaN
54,service JOMR,35831,36414,13,7,1,0,45.0
55,service JOMR,36898,37165,14,7,1,0,46.0


In [16]:
def score_checker(df_points:pd.DataFrame) -> dict:
    
    """
    Fonction pour vérifier la cohérence des score par rapport aux side switch.
    Après un *SWITCH*, la somme des colonnes '_score' doit être égale à un multiple de 5 ou 7 (selon le nombre de points par set)
    Elle indique le format du match (15 ou 21 points par set) selon le multiple trouvé.
    Elle indique également le score final, avec le détail par set, pour les deux équipes.
    Si une incohérence est détectée, elle retourne un message d'erreur indiquant le problème.

    Arg :
        df_points: DataFrame contenant les segments de points extraits, avec les colonnes : 'point_index','action','start_frame','end_frame','score_team1','score_team2','set_team1','set_team2'
    Returns:
        dict : Dictionnaire contenant les informations sur le format du match et le score final
    """
    recap_dict = dict({
        'match_format': None,
        'victoire': None,
        'final_score': None,
        'score_by_set': [],
    })
    
    # Retirer les lignes correspondant aux temps morts
    temp_df = df_points[df_points['Service_side'] != 'Temps mort'].reset_index(drop=True)

    # Récupérer les noms des équipes à partir des colonnes du DataFrame
    team1_name = temp_df.columns[3].replace('_score', '')
    team2_name = temp_df.columns[4].replace('_score', '')
    # Initialiser les variables pour le score et le format du match
    score_switch_points = []
    for idx, row in temp_df.iterrows():
        if row['Service_side'] == '*SWITCH*':
            if idx + 1 < len(temp_df):
                score_sum = temp_df.at[idx + 1, f'{team1_name}_score'] + temp_df.at[idx + 1, f'{team2_name}_score']
            else:
                score_sum = row[f'{team1_name}_score'] + row[f'{team2_name}_score']
            score_switch_points.append(score_sum)

    # print("Scores au moment des switchs :", score_switch_points)

    # Vérifier les multiples de 5 ou 7
    multiples_of_5 = all(score % 5 == 0 for score in score_switch_points)
    multiples_of_7 = all(score % 7 == 0 for score in score_switch_points)
    if multiples_of_5:
        match_format = "15 points par set"
    elif multiples_of_7:
        match_format = "21 points par set"
    else:
        return {"message": "Incohérence détectée : les scores au moment des switch ne sont pas des multiples de 5 ou 7."}
    
    # Récupérer le score final et le détail par set
    recap_dict['match_format'] = match_format
    final_score_team1 = temp_df[f'{team1_name}_sets'].iloc[-1]
    final_score_team2 = temp_df[f'{team2_name}_sets'].iloc[-1]
    recap_dict['final_score'] = f"{final_score_team1} - {final_score_team2}"
    
    # Déterminer le gagnant
    if final_score_team1 > final_score_team2:
        recap_dict['victoire'] = team1_name
    elif final_score_team2 > final_score_team1:
        recap_dict['victoire'] = team2_name
    else:
        recap_dict['victoire'] = "Égalité"

    # Détail par set
    set_count = 0
    current_set_scores = []
    for idx, row in temp_df.iterrows():
        if row['Service_side'] == 'debut du set' and idx > 0:
            current_set_scores.append({
                'set': set_count + 1,
                'score': f"{row[f'{team1_name}_score']} - {row[f'{team2_name}_score']}"
            })
            set_count += 1
        elif idx == len(temp_df) - 1:  # Dernière ligne du DataFrame
            current_set_scores.append({
                'set': set_count + 1,
                'score': f"{row[f'{team1_name}_score']} - {row[f'{team2_name}_score']}"
            })

    recap_dict['score_by_set'] = current_set_scores

    return recap_dict
    
######------------------------------------------------------------------------------------------------
score_dict = score_checker(test_df_points)
score_dict




{'match_format': '15 points par set',
 'victoire': 'JOMR',
 'final_score': '1 - 0',
 'score_by_set': [{'set': 1, 'score': '0 - 0'}, {'set': 2, 'score': '14 - 7'}]}